In [3]:
import numpy as np
import pandas as pd

In [4]:
pv = pd.read_csv("../data/test_set4/pv_test_set4.csv").iloc[:,2]
demand = pd.read_csv("../data/test_set4/demand_test_set4.csv").iloc[:,1]

forecast_profile =  pd.read_csv("../data/profiles/storage_schedule_forecast_v1.csv")

# last_week_profile =  pd.read_csv("../data/profiles/storage_schedule_last_week.csv")

In [5]:
def produce_score(pv, demand, profile):
    # initial the B values from the profiles
    B_d_k = profile["B"].values.reshape(7,48)
    
    # the demand and the total pv generation come from the real next week data
    L_d_k = demand.values.reshape(7,48)
    Ptotal_d_k = pv.values.reshape(7,48)
    
    # The power charged into the battery is the minimal value of the profile B value and the real total pv generation value
    P_d_k = np.zeros([7,31])
    for d in range(7):
        for k in range(31):
            P_d_k[d,k] = min(B_d_k[d,k],Ptotal_d_k[d,k])
    
    # the score for each day
    S_d = np.zeros(7)
    for d in range(7):
        # Pratio_d is the ratio of the power that is charged into battery, coming from the PV generation in a day
        Pratio_d = sum(P_d_k[d,k] for k in range(31))/sum(B_d_k[d,k] for k in range(31))
        
        # the initial peak demand is the maximul demand in the evening time of a day
        Peakold_d = max(L_d_k[d,k] for k in range(31,42))
        # the current peak demand is the maximul of demand subtracting the power discharging from the battery  (B is negetive)in the evening time of a day
        Peaknew_d = max(L_d_k[d,k]+B_d_k[d,k] for k in range(31,42))
        # the ratio how much the peak demand reducing
        RPeak_d = 100*(Peakold_d-Peaknew_d)/Peakold_d
        
        # the score for a day is the combine of the two ratio
        S_d[d] = RPeak_d*(3*Pratio_d+1*(1-Pratio_d))
    score = S_d.mean()   
    return score

In [6]:
# using the forecast profile to get the score
produce_score(pv, demand, forecast_profile)

114.01535119130865

In [5]:
# using the last_week profile to get the score
produce_score(pv, demand, last_week_profile)

105.93314477950734